# 1. pan-ALL

## 1.1 pan-ALL snvindels

In [2]:
#generate a table for vcf generation 
#input is snvindels from PAN-ALL paper supplimentary table. file name: "snvindel"
#output is a table with the following columns
#sample_name     chr     pos     ref     alt     wgstumoralt     wgstumorref     wgsnoralt       wgsnorref       westumoralt     westumorref     wesnoralt       wesnorref       dna_assay
#require the python package called liftover

import sys
import re
import os
from liftover import get_lifter

#liftover
converter = get_lifter('hg19','hg38')

HG19FastFile = '/research/rgs01/resgen/legacy/gb_customTracks/tp/genomes/hg19.gz'

#indel coordinate correction
def GETREFALT(chro,ref,alt,pos,fa):
        ref = ref.replace('-','')
        alt = alt.replace('-','')
        p = str(int(pos) - 1)
        chro = chro if chro.startswith('chr') else 'chr'+chro
        AddBase = list(os.popen('samtools faidx '+fa+' '+chro+':'+p+'-'+p))[1].strip().upper()
        return AddBase+ref,AddBase+alt,p
#hg38 coordinate convertion
def COVHG38(orgchr,liftoverout):
    pos = ''
    for e in liftoverout:
        hg38chr,hg38pos,hg38std = e
        hg38chr = hg38chr[3:] if hg38chr[:3].lower() == 'chr' else hg38chr
        if hg38chr == orgchr:
            pos = str(hg38pos)
    return pos
    

#with the following columns
#sample_name     chr     pos     ref     alt     wgstumoralt     wgstumorref     wgsnoralt       wgsnorref       westumoralt     westumorref     wesnoralt       wesnorref       dna_assay
fh = open('snvindel')
out = open('panallsnvindel','w')
out.write('\t'.join(['sample_name','chr','pos','ref','alt','wgstumoralt','wgstumorref','wgsnoralt','wgsnorref','westumoralt','westumorref','wesnoralt','wesnorref','dna_assay'])+'\n')
for line in fh:
    l = line.strip().split('\t')
    chrom = l[4]
    chrom = chrom[3:] if chrom[:3].lower() == 'chr' else chrom
    pos = l[5]
    ref = l[6].replace('-','').strip()
    alt = l[7].replace('-','').strip()
    if not ref or not alt:
        ref,alt,pos = indelcor.GETREFALT(chrom,ref,alt,pos,fasta)
    hg38Cor = converter[chrom][int(pos)]
    pos = COVHG38(chrom,hg38Cor)
    if not pos:
        continue
    OUTL = [l[1],chrom,pos,ref,alt]
    ASSAY = ''
    #WGS read count
    if re.match("\d+$",l[15]):
        OUTL.append(l[15])
        OUTL.append(str(int(l[16])-int(l[15])))
        if re.match("\d+$",l[17]):
            OUTL.append(l[17])
            OUTL.append(str(int(l[18])-int(l[17])))
        else:
            OUTL.extend(['']*2)
        ASSAY = 'WGS'
    else:
        OUTL.extend(['']*4)
    
    if re.match("\d+$",l[19]):
        OUTL.append(l[19])
        OUTL.append(str(int(l[20])-int(l[19])))
        if re.match("\d+$",l[21]):
            OUTL.append(l[21])
            OUTL.append(str(int(l[22])-int(l[21])))
        else:
             OUTL.extend(['']*2)
        if not ASSAY:
            ASSAY = 'WES'
    else:
        OUTL.extend(['']*4)
    OUTL.append(ASSAY)
    out.write('\t'.join(OUTL)+'\n')
fh.close()
out.close()


ModuleNotFoundError: No module named 'liftover'

### functions used to generate vcf file

In [5]:
%%writefile snvindel.py 

import os
import json


#generate SNVindel data store file for generating vcf file or combining of multiple file
#infile: input file tab delimited
#outfile: output file
#return all samples in input file
#optattr: optional attribute such as pmid, vorigin...
def GENSNVINDEL(infile,outfile,optattr=[]):
        fh = open(infile)
        SNVINDEL = {}
        HEAD = fh.readline().strip().split('\t')
        samidx = HEAD.index('sample_name')
        sample = set()
        chridx = HEAD.index('chr')
        posidx = HEAD.index('pos')
        refidx = HEAD.index('ref')
        altidx = HEAD.index('alt')
        rck = ['wgstumoralt','wgstumorref','wgsnoralt','wgsnorref','westumoralt','westumorref','wesnoralt','wesnorref','rnatumoralt','rnatumorref']
        rckidx = {x:HEAD.index(x) for x in rck if x in HEAD}
        rckused = list(rckidx.keys())
        if optattr:
            optattridx = {x.lower():HEAD.index(x) for x in optattr if x in HEAD}
        for line in fh:
            L = line.replace('\n','').split('\t')
            sample.add(L[samidx])
            chron,pos,ref,alt = [L[x] for x in [chridx,posidx,refidx,altidx]]
            rcv = [L[rckidx[x]] for x in rckused]
            readic = dict(zip(rckused,rcv))
            ID,JS = PARSNVINDEL(genomefa,'',[chron,pos,ref,alt],readic)
            if optattr:
                for attr in optattr:
                    JS[attr.lower()] = L[optattridx[attr.lower()]]
            if ID not in SNVINDEL:
                SNVINDEL[ID] = {L[samidx]:JS}
            else:
                SNVINDEL[ID].update({L[samidx]:JS})
        fh.close()
        snvout = open(outfile,'w')
        for ID in SNVINDEL:
            snvout.write('\t'.join([ID,json.dumps(SNVINDEL[ID])])+'\n')
        snvout.close()
        return sample

# vtype could be somatic or germline
#the parameters snvindel a list include chromsome, position, reference allele and alternative allele
#will remove '-' from reference or alternative allele
#retrive nucleotide and coordinate before indel
#readcount is a dictionary including any of following keys:
#WGS:   wgstumoralt,wgstumorref,wgsnoralt,wgsnorref,
#CGI:   cgitumoralt,cgitumorref,cginoralt,cginorref,
#WES:   westumoralt,westumorref,wesnoralt,wesnorref,
#validation:    cctumoralt,cctumorref,ccnoralt,ccnorref,
#RNA:   rnatumoralt,rnatumorref,rnanoralt,rnanorref 
def snvindel(vtype,snvindel,readcount):
        chrom,pos,ref,alt = snvindel
        chrom = chrom[3:] if chrom[:3].lower() == 'chr' else chrom
        chrom = corectchr.CORECTCHR(chrom)
        JS = PARSEREADCOUNT(vtype,readcount)
        ID = '\t'.join([chrom,pos,ref,alt])
        return ID,JS

def PARSEREADCOUNT(vartype,rc):
     k2v = {'wgstumoralt':'tumor_DNA_WGS_alt','wgstumorref':'tumor_DNA_WGS_ref','wgsnoralt':'germline_DNA_WGS_alt','wgsnorref':'germline_DNA_WGS_ref',
               'cgitumoralt':'tumor_DNA_CGI_alt','cgitumorref':'tumor_DNA_CGI_ref','cginoralt':'germline_DNA_CGI_alt','cginorref':'germline_DNA_CGI_ref',
               'westumoralt':'tumor_DNA_WES_alt','westumorref':'tumor_DNA_WES_ref','wesnoralt':'germline_DNA_WES_alt','wesnorref':'germline_DNA_WES_ref',
               'cctumoralt':'tumor_DNA_CC_alt','cctumorref':'tumor_DNA_CC_ref','ccnoralt':'germline_DNA_CC_alt','ccnorref':'germline_DNA_CC_ref',
               'rnatumoralt':'tumor_RNA_alt','rnatumorref':'tumor_RNA_ref','rnanoralt':'germline_RNA_alt','rnanorref':'germline_RNA_ref',}
    k = [[['wgstumoralt','wgstumorref'],['wgsnoralt','wgsnorref']],[['cgitumoralt','cgitumorref'],['cginoralt','cginorref']],[['westumoralt','westumorref'],['wesnoralt','wesnorref']],
                [['cctumoralt','cctumorref'],['ccnoralt','ccnorref']],[['rnatumoralt','rnatumorref'],['rnanoralt','rnanorref']]]
    if vartype:
        JSINFO = {'vorigin':vartype}
    else:
        JSINFO = {}
    for t,g in k:
        tval = GETREADCOUNT(t,rc)
        gval = GETREADCOUNT(g,rc)
        if tval:
            for i in [0,1]:
                JSINFO[k2v[t[i]]] = tval[i]
            if gval:
                for i in [0,1]:
                    JSINFO[k2v[g[i]]] = gval[i]
    if 'tumor_DNA_WGS_alt' in JSINFO:
        JSINFO['dna_assay'] = 'wgs'
    elif 'tumor_DNA_CGI_alt' in JSINFO:
        JSINFO['dna_assay'] = 'cgi'
    elif 'tumor_DNA_WES_alt' in JSINFO:
        JSINFO['dna_assay'] = 'wes'
    return JSINFO
    
#used in PARSEREADCOUNT
def GETREADCOUNT(k,rc):
        rtv = False
        k1,k2 = k
        if k1 in rc and k2 in rc and str(rc[k1]) and str(rc[k2]):
                k1val = int(rc[k1])
                k2val = int(rc[k2])
                if k1val != 0 or k2val != 0:
                        rtv = [k1val,k2val]
        return rtv
def CORECTCHR(c):
        CHRD = {'23':'X','24':'Y','MT':'M','chr23':'chrX','chr24':'chrY','chrMT':'chrM'}
        if c in CHRD:
                return CHRD[c]
        else:
                return c

            
#convert snvindel data-store file to vcf file
#input: snvindel data-store file
#       vcf header file
#output vcf file
def DATASTORE2VCF(dsfile,headerfile,outputvcf):
    out = open(outputvcf,'w')
    headerfh = open(headerfile)
    for i in headerfh:
        out.write(i.strip()+'\n')
    headerfh.close()
    SAMPLES = set()
    VARSamVal = {}
    fh = open(dsfile)
    for line in fh:
        line = line.replace('\n','')
        lineL = line.split('\t')
        JS = json.loads(lineL[4])
        ID = '\t'.join(lineL[0:4])
        FormatSamVal,rtSamples = PARSEJS(JS)
        VARSamVal[ID] = FormatSamVal
        SAMPLES.update(rtSamples)
    fh.close()
    SAMPLES = list(SAMPLES)
    out.write('\t'.join(['#CHROM','POS','ID','REF','ALT','QUAL','FILTER','INFO','FORMAT']+SAMPLES)+'\n')
    for id in VARSamVal:
        if len(VARSamVal[id]) < 3:
            print(id,VARSamVal[id])
            continue
        outformat = VARSamVal[id][0]
        outsams = VARSamVal[id][1]
        outvals = VARSamVal[id][2]
        chron,pos,REF,ALT = id.split('\t')
        formatLength = outformat.split(':')
        outValList = [':'.join(['.']*len(formatLength))] * len(SAMPLES)
        for x,s in enumerate(outsams):
            IDX = SAMPLES.index(s)
            outValList[IDX] = outvals[x]
        outlist = [chron,pos,'.',REF,ALT,'.','.','.',outformat] + outValList
        out.write('\t'.join(outlist)+'\n')
    out.close()
        
def PARSEJS(samvaljs):
        SamVal = []
        rtSamples = GETSAMPLE(samvaljs.keys()) #function GETSAMPLE
        PARSEJS_FORMAT_List = SETFORMAT(samvaljs)
        PARSEJS_FORMAT = ':'.join(PARSEJS_FORMAT_List)
        SamVal.append(PARSEJS_FORMAT)
        for s in samvaljs:
                samvallist = []
                for attrkey in PARSEJS_FORMAT_List:
                        if 'tumor' in attrkey or 'germline' in attrkey:
                                if attrkey+'_ref' in samvaljs[s] and attrkey+'_alt' in samvaljs[s]:
                                        samvallist.append(str(samvaljs[s][attrkey+'_ref'])+','+str(samvaljs[s][attrkey+'_alt']))
                                elif attrkey+'_ref' in samvaljs[s] or attrkey+'_alt' in samvaljs[s]:
                                        print('only have reads count for ref or alt: ',file=sys.stderr)
                                else:
                                        samvallist.append('.')
                        else:
                                if attrkey in samvaljs[s]:
                                        samvallist.append(samvaljs[s][attrkey])
                                else:
                                        samvallist.append('.')
                if len(SamVal) == 1:
                        SamVal.append([s])
                        SamVal.append([':'.join(samvallist)])
                elif len(SamVal) > 1:
                        SamVal[1].append(s)
                        SamVal[2].append(':'.join(samvallist))
                else:
                        print('no format generated for:',file=sys.stderr)
        return SamVal,rtSamples
def GETSAMPLE(sams):
    RTSAM = set()
    for s in sams:
        RTSAM.add(s)
    return RTSAM

#get final format for vcf file 
def SETFORMAT(fjs):
        setformat_format = ['tumor_DNA_WGS',\
                        'germline_DNA_WGS',\
                        'tumor_DNA_CGI',\
                        'germline_DNA_CGI',\
                        'tumor_DNA_WES',\
                        'germline_DNA_WES',\
                        'tumor_DNA',\
                        'germline_DNA',\
                        'tumor_DNA_CC',\
                        'germline_DNA_CC',\
                        'tumor_RNA',\
                        'dna_assay',\
                        'pantargetsignature',\
                        'tcgaskcmsignature',\
                        'project',\
                        'vorigin',\
                        'pmid']
        allkey = set()
        for sam in fjs.keys():
                for k in fjs[sam]:
                        if 'tumor' in k or 'germline' in k:
                                testk = k[:-4]
                        else:
                                testk = k
                        allkey.add(testk)
        format_list = []
        for fk in setformat_format:
                if fk in allkey:
                        format_list.append(fk)
        return format_list

    

Overwriting snvindel.py


In [ ]:
%%writefile VCFheader
##fileformat=VCFv4.2
##FILTER=<ID=PASS,Description="All filters passed">
##FORMAT=<ID=tumor_DNA_WGS,Number=R,Type=Integer,Description="Allelic depths for the ref and alt alleles in the order listed, tumor WGS">
##FORMAT=<ID=germline_DNA_WGS,Number=R,Type=Integer,Description="Allelic depths for the ref and alt alleles in the order listed, germline WGS">
##FORMAT=<ID=tumor_DNA_WES,Number=R,Type=Integer,Description="Allelic depths for the ref and alt alleles in the order listed, tumor WES">
##FORMAT=<ID=germline_DNA_WES,Number=R,Type=Integer,Description="Allelic depths for the ref and alt alleles in the order listed, germline WES">
##FORMAT=<ID=dna_assay,Number=1,Type=String,Description="Source of the variant generated from, can be cgi,wgs,wes">
##contig=<ID=1,length=248956422>
##contig=<ID=2,length=242193529>
##contig=<ID=3,length=198295559>
##contig=<ID=4,length=190214555>
##contig=<ID=5,length=181538259>
##contig=<ID=6,length=170805979>
##contig=<ID=7,length=159345973>
##contig=<ID=8,length=145138636>
##contig=<ID=9,length=138394717>
##contig=<ID=10,length=133797422>
##contig=<ID=11,length=135086622>
##contig=<ID=12,length=133275309>
##contig=<ID=13,length=114364328>
##contig=<ID=14,length=107043718>
##contig=<ID=15,length=101991189>
##contig=<ID=16,length=90338345>
##contig=<ID=17,length=83257441>
##contig=<ID=18,length=80373285>
##contig=<ID=19,length=58617616>
##contig=<ID=20,length=64444167>
##contig=<ID=21,length=46709983>
##contig=<ID=22,length=50818468>
##contig=<ID=X,length=156040895>
##contig=<ID=Y,length=57227415>
##contig=<ID=M,length=16569>

In [ ]:
#generate vcf file
import snvindel

#generate data store file
optattr = ['dna_assay']
infile = 'panallsnvindel'
outfile = 'panall.hg38.datastore'
snvindel.GENSNVINDEL(infile,outfile,optattr=optattr)

#generate vcf file
vcfinput = 'panall.hg38.datastore'
vcfheader = 'VCFheader'
vcfoutfile = 'panall.hg38.vcf'
snvindel.DATASTORE2VCF(vcfinput,vcfheader,vcfoutfile)


## 1.2 pan-ALL sv

### functions to generate svfusion file

In [ ]:
%%writefile svfusion.py 

#!/usr/bin/python3

import sys,os
import json
from decimal import Decimal
    
#infile: input file 
#outfile: output file
#assay: data type. could be fusion, cnv, loh and sv
#optattr: attributes which is optional. keys can be anything including 'vorigin', 'dna_assay' and 'pmid'
#return all samples in input file 
def GENVARFH(infile,outfile,assay,optattr={}):
        fh = open(infile)
        out = open(outfile,'w')
        HEAD = fh.readline().strip().split('\t')
        samidx = HEAD.index('sample_name')
        sample = set()
        if optattr:
                optkey = list(optattr.keys())
                HEAD.extend(optkey)
                optval = [optattr[x] for x in optkey]
        else:
                optkey = []
        for line in fh:
                L = line.replace('\n','').split('\t')
                sample.add(L[samidx])
                if optkey:
                        L.extend(optval)
                RT = SVFUSION(assay,HEAD,optkey,L)
                out.write(RT)
        out.close()
        fh.close()
        return sample

#vtype: 'sv' or 'fusion'
#vhead svhead = ['sample_name', 'gene_a', 'chr_a', 'position_a', 'strand_a', 'gene_b', 'chr_b', 'position_b','strand_b','vorigin','dna_assay','manual_review']
#voptatrr svoptatrr = ['vorigin','dna_assay','manual_review']
#valuelist value should correspond to vhead
def SVFUSION(vtype,vhead,voptatrr,valuelist):
        if vtype == 'sv':
                n = 5
        elif vtype == 'fusion':
                n = 2
        return PARSEFUSV(valuelist,vhead,voptatrr,n)

################
#SV & Fusion
def PARSEFUSV(L,headL,optattr,dt):
        ca = L[headL.index('chr_a')]
        ca = 'chr'+ca[3:] if ca[0:3].lower() == 'chr' else 'chr'+ca
        cb = L[headL.index('chr_b')]
        cb = 'chr'+cb[3:] if cb[0:3].lower() == 'chr' else 'chr'+cb
        psa = L[headL.index('position_a')]
        psa = POSMIN1(psa)
        psb = L[headL.index('position_b')]
        psb = POSMIN1(psb)
        stda = L[headL.index('strand_a')]
        stdb = L[headL.index('strand_b')]
        if 'gene_a' in headL:
                ga = L[headL.index('gene_a')].strip()
                ga = CHANGENENAME(ga)
        else:
                ga = False
        if 'gene_b' in headL:
                gb = L[headL.index('gene_b')].strip()
                gb = CHANGENENAME(gb)
        else:
                gb = False
        sam = L[headL.index('sample_name')]
        JSA = {'dt':dt,'strandA':stda,'strandB':stdb,'sample':sam}
        if 'svtype' in headL:
                svtype = L[headL.index('svtype')]
                if svtype:
                        JSA['svtype'] = svtype
        readList = ['clipreadA','clipreadB','totalreadA','totalreadB']
        for readk in readList:
                if readk in headL:
                        readv = L[headL.index(readk)]
                        if readv and readv.isdigit():
                                readv = int(readv)
                                if readv == 0:
                                        continue
                                JSA[readk] = readv
        if ga:
                JSA['geneA'] = ga
        if gb:
                JSA['geneB'] = gb
        mattr = PARSEMATTR(L,headL,optattr)
        if mattr:
                JSA['mattr'] = mattr
        JSB = JSA.copy()
        JSA['chrB'] = cb
        JSA['posB'] = int(psb)
        JSB['chrA'] = ca
        JSB['posA'] = int(psa)
        return '\t'.join([ca,psa,psa,json.dumps(JSA,sort_keys=True)])+'\n'+'\t'.join([cb,psb,psb,json.dumps(JSB,sort_keys=True)])+'\n'

def POSMIN1(p):
        return str(int(Decimal(p))-1)

def CHANGENENAME(g):
        if g == 'NA' or ',' in g:
                return False
        else:
                return g
    

In [7]:
#generate mds sv file
#input is sv table from PAN-ALL paper supplimentary table. file name: "sv"
#require the python package called liftover

import sys
import re
import os
from liftover import get_lifter
import svfusion

#hg38 coordinate convertion
def COVHG38(orgchr,liftoverout):
    pos = ''
    for e in liftoverout:
        hg38chr,hg38pos,hg38std = e
        hg38chr = hg38chr[3:] if hg38chr[:3].lower() == 'chr' else hg38chr
        if hg38chr == orgchr:
            pos = str(hg38pos)
    return pos

#liftover
converter = get_lifter('hg19','hg38')

fh = open('sv')
out = open('panallsv','w')
out.write('\t'.join(['chr_a','chr_b','position_a','position_b','strand_a','strand_b','gene_a','gene_b','sample_name','svtype'])+'\n')

for line in fh:
    l = line.strip().split('\t')
    OUTL = []
    sample = l[1]
    chroma = l[2]
    chroma = chroma if chroma[:3].lower() == 'chr' else 'chr'+chroma
    chromb = l[5]
    chromb = chromb if chromb[:3].lower() == 'chr' else 'chr'+chromb
    posa = l[3]
    stda = l[4]
    posb = l[6]
    stdb = l[7]
    svt = l[8]
    hg38Cora = converter[chroma][int(posa)]
    posa = COVHG38(chroma,hg38Cora)
    if not posa:
        continue
    hg38Corb = converter[chromb][int(posb)]
    posb = COVHG38(chromb,hg38Corb)
    if not posb:
        continue
    if l[13]:
        if ',' in l[13]:
            ga = l[13].strip().split(',')[0]
        else:
            ga = l[13]
    else:
        ga = ''
    if l[14]:
        if ',' in l[14]:
            gb = l[14].strip().split(',')[0]
        else:
            gb = l[14]
    else:
        gb = ''
    OUTL = [chroma,chromb,posa,posb,stda,stdb,ga,gb,sample,svt]
    out.write('\t'.join(OUTL)+'\n')
out.close()
fh.close()

#gernate mds sv file
svfusion('panallsv','panall.hg38.sv','sv')
    

ModuleNotFoundError: No module named 'liftover'

## 1.3 pan-ALL fusion

In [ ]:
#pan-ALL fusion
#coordinate can not be found from pan-ALL paper supplementary table
#fusion data was extracted from pediatric data plus some data from kohei.
#current fusion data only avilable for some of pan-ALL samples not all
#fusion file 'panall.hg38.fusion'

